## linkes for the project

[![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow.svg)](https://universe.roboflow.com/capstoneteam/road-lines-segmentation-2/dataset/1)

[![GitHub Repoistory](https://badges.aleen42.com/src/github.svg)](https://github.com/TariqAlhathloul/Smart-Street)


## installation

In [1]:
!pip install ultralytics -q
!pip install roboflow -q

In [2]:
from ultralytics import YOLO
from roboflow import Roboflow
import pandas as pd 
import os

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
API_KEY = user_secrets.get_secret("Roboflow_API_KEY")

## upload the dataset

In [3]:
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("capstoneteam").project("road-lines-segmentation-2")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.97, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Road-Lines-Segmentation-2-1 in yolov8:: 100%|██████████| 18696/18696 [00:02<00:00, 7905.11it/s]


In [4]:
TRAIN_DIR_IAMAGES = '/kaggle/working/Road-Lines-Segmentation-2-1/train/images'
VAL_DIR_IAMGES = '/kaggle/working/Road-Lines-Segmentation-2-1/valid/images'
TEST_DIR_IMAGES = '/kaggle/working/Road-Lines-Segmentation-2-1/test/images'

In [5]:
print(f"Total Training: {len(os.listdir(TRAIN_DIR_IAMAGES))} images")
print(f"Total Validation: {len(os.listdir(VAL_DIR_IAMGES))} images")
print(f"Total Testing: {len(os.listdir(TEST_DIR_IMAGES))} images")

Total Training: 6849 images
Total Validation: 1598 images
Total Testing: 895 images


## check if the GPU is avalabie

In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 19 15:19:39 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P0             27W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## upload the model

In [6]:
#model = YOLO('/kaggle/input/best/pytorch/default/1/best.pt')
# first experiment using the pervious model from the mini project

In [7]:
# second experiment usning yolov8n-eg
model = YOLO('yolov8n-seg.yaml').load('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 119MB/s]

Transferred 355/417 items from pretrained weights


## overwrite the data.yaml file

In [8]:
%%writefile /kaggle/working/Road-Lines-Segmentation-2-1/data.yaml
names:
- bus
- car
- license_plate
- solid-yellow-line
- truck
nc: 5
roboflow:
  license: CC BY 4.0
  project: road-lines-segmentation-2
  url: https://universe.roboflow.com/capstoneteam/road-lines-segmentation-2/dataset/1
  version: 1
  workspace: capstoneteam
test: /kaggle/working/Road-Lines-Segmentation-2-1/test/images
train: /kaggle/working/Road-Lines-Segmentation-2-1/train/images
val: /kaggle/working/Road-Lines-Segmentation-2-1/valid/images



Overwriting /kaggle/working/Road-Lines-Segmentation-2-1/data.yaml


## Hyperparameters, finetuning

In [9]:
IMG_SIZE = 640
BATCH_SIZE = 32
EPOCHS = 100

In [10]:
results = model.train(
    data = '/kaggle/working/Road-Lines-Segmentation-2-1/data.yaml',
    task = 'segment',
    imgsz = IMG_SIZE,
    epochs = EPOCHS,
    batch = -1,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    val = True,
    name = 'Segment-Model',
    project = 'experiment-1',
    patience = 10,
    save = True,
    optimizer = 'auto'
)

Ultralytics YOLOv8.2.97 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.yaml, data=/kaggle/working/Road-Lines-Segmentation-2-1/data.yaml, epochs=100, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=experiment-1, name=Segment-Model, exist_ok=False, pretrained=yolov8n.pt, optimizer=auto, verbose=True, seed=93, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fal

100%|██████████| 755k/755k [00:00<00:00, 20.2MB/s]
2024-09-19 15:22:36,857	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-09-19 15:22:37,422	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


...

                   all       1598       6493      0.678      0.579      0.622      0.512      0.644      0.558      0.592      0.428
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 54, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



64 epochs completed in 2.155 hours.
Optimizer stripped from experiment-1/Segment-Model/weights/last.pt, 6.8MB
Optimizer stripped from experiment-1/Segment-Model/weights/best.pt, 6.8MB

Validating experiment-1/Segment-Model/weights/best.pt...
Ultralytics YOLOv8.2.97 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,259,039 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:18<00:00,  1.34it/s]


                   all       1598       6493       0.66      0.576       0.63      0.523      0.675      0.545      0.611       0.44
                   bus        191        192      0.897      0.633      0.798      0.719      0.902      0.589      0.798      0.686
                   car       1471       4309      0.868      0.853      0.902      0.765      0.886      0.818      0.895      0.649
         license_plate        148        225      0.579      0.196      0.328       0.18       0.52      0.138      0.231      0.071
     solid-yellow-line       1559       1714      0.762      0.936      0.933      0.804      0.822      0.937      0.937      0.649
                 truck         45         53      0.195      0.264      0.189      0.149      0.247      0.245      0.193      0.147
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to experiment-1/Segment-Model


lr/pg0,▁▅███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁
lr/pg1,▁▅███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁
lr/pg2,▁▅███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁
metrics/mAP50(B),▁▄▃▆▅▆▆▇▄▆▇▇▇▆▇▆▇▆▇█▆▇█▇▇▇▇████▇███▇▇▇▇█
metrics/mAP50(M),▁▄▃▆▅▆▆▇▄▆▇▇▆▆▇▆▇▆▆▇▆▇█▇▆▆▇█▇██▆▇█▇▇▆▇▇█
metrics/mAP50-95(B),▁▃▂▄▄▅▅▅▃▅▆▆▆▅▆▅▇▆▆▇▆▇█▆▇▇▇████▇████▇▇██
metrics/mAP50-95(M),▁▃▃▅▄▅▅▆▃▆▆▆▆▆▆▆▇▆▆▇▆▇█▇▇▇▇████▆████▇▇██
metrics/precision(B),█▅▆▃▂▂▂▆▁▄▃▄▃▃▄▂▃▂▃▃▃▄▃▃▃▂▃▄▄▄▄▄▅▃▃▃▃▃▃▃
metrics/precision(M),█▅▆▄▂▂▂▅▁▄▃▄▂▃▄▂▃▃▂▃▃▃▃▃▃▂▄▄▃▄▄▄▄▄▃▃▃▃▃▄
metrics/recall(B),▁▃▃▅▅▆▇▅▄▅▇▇▆█▆▆█▅▆█▆███▇▆██▇██▅▆▇▇▆▆▆▆▇
metrics/recall(M),▁▃▃▅▅▆▇▄▃▅▇▆▆█▆▅█▅▆▇▆▇█▇▆▅▇█▇▇▇▅▆▆▇▅▆▆▆▆


## evaluation on the validationset


In [15]:
evaluation = model.val(
    data = '/kaggle/working/Road-Lines-Segmentation-2-1/data.yaml',
    task = 'segment',
    imgsz = 640,
    iou = 0.6,
    batch = 32,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    name = 'Model-segment-evaluation',
    save_json = True
)

Ultralytics YOLOv8.2.97 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,259,039 parameters, 0 gradients, 12.0 GFLOPs


val: Scanning /kaggle/working/Road-Lines-Segmentation-2-1/valid/labels.cache... 1598 images, 5 backgrounds, 0 corrupt: 100%|██████████| 1598/1598 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/Road-Lines-Segmentation-2-1/valid/images/front-left-NO20240903-145041-000844F_MP4-0244_jpg.rf.e18ae73373b873bc48c4e2bdedcc1403.jpg: 1 duplicate labels removed



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/50 [00:00<?, ?it/s]

requirements: Ultralytics requirement ['pycocotools>=2.0.6'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 11.8 MB/s eta 0:00:00a 0:00:01

requirements: AutoUpdate success ✅ 12.8s, installed 1 package: ['pycocotools>=2.0.6']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:56<00:00,  1.14s/it]


                   all       1598       6493      0.665      0.577      0.634      0.529      0.667      0.577      0.629      0.431
                   bus        191        192      0.897      0.635      0.799      0.723      0.897      0.633      0.799      0.688
                   car       1471       4309      0.868      0.854      0.903       0.77      0.868      0.854      0.902      0.667
         license_plate        148        225       0.58      0.196      0.344      0.193      0.585      0.196      0.314      0.104
     solid-yellow-line       1559       1714      0.783      0.936      0.935      0.807      0.786      0.938      0.935      0.547
                 truck         45         53      0.199      0.264       0.19      0.151        0.2      0.264      0.196      0.149
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 0.8ms postprocess per image
Saving experiment-1/Model-segment-evaluation/predictions.json...
Results saved to experiment-1/Model-segment-evaluation


## predictions on the testset

In [17]:
predictions = model.predict(
    data = '/kaggle/working/Road-Lines-Segmentation-2-1/data.yaml',
    source = '/kaggle/working/Road-Lines-Segmentation-2-1/test/images',
    task = 'segment',
    imgsz = 640,
    device = 0,
    verbose = True,
    seed = 93,
    plots = True,
    name = 'Model-segment-predictions',
    save_json = True,
)


image 1/895 /kaggle/working/Road-Lines-Segmentation-2-1/test/images/NO20240828-152333-000152F_MP4-0008_jpg.rf.9502c63e6fca67ce35acbe09ec9b608b.jpg: 640x640 9 cars, 9.4ms
image 2/895 /kaggle/working/Road-Lines-Segmentation-2-1/test/images/NO20240828-152333-000152F_MP4-0075_jpg.rf.17ae937a53e26d9180122e1fc93433b9.jpg: 640x640 5 cars, 7.4ms
image 3/895 /kaggle/working/Road-Lines-Segmentation-2-1/test/images/NO20240828-152333-000152F_MP4-0078_jpg.rf.f4c6b90fdedd460e7f43a435a9e849cc.jpg: 640x640 5 cars, 7.5ms
image 4/895 /kaggle/working/Road-Lines-Segmentation-2-1/test/images/NO20240828-152333-000152F_MP4-0079_jpg.rf.ee2be7d4d588dd336f1bdfbe7d213256.jpg: 640x640 5 cars, 7.5ms
image 5/895 /kaggle/working/Road-Lines-Segmentation-2-1/test/images/NO20240828-152333-000152F_MP4-0081_jpg.rf.105bfbe1460cf7cc9906ab385f983a22.jpg: 640x640 5 cars, 1 solid-yellow-line, 7.1ms
image 6/895 /kaggle/working/Road-Lines-Segmentation-2-1/test/images/NO20240828-152333-000152F_MP4-0083_jpg.rf.83d566c00bbaadb772

## apply quantization

In [20]:
model.export(format="onnx", int8=True, dynamic=True, simplify=True, data='/kaggle/working/Road-Lines-Segmentation-2-1/data.yaml')

Ultralytics YOLOv8.2.97 🚀 Python-3.10.14 torch-2.4.0 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'experiment-1/Segment-Model/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 41, 8400), (1, 32, 160, 160)) (6.5 MB)
requirements: Ultralytics requirements ['onnxslim==0.1.34', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 248.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 196.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 246.6 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 19.4s, installed 2 packages: ['onnxslim==0.1.34', 'onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.2 opset 19...
ONNX: slimming with onnxslim 0.1.34...
ONNX: export success ✅ 52.2s, 

'experiment-1/Segment-Model/weights/best.onnx'

In [34]:
import shutil

shutil.make_archive('/kaggle/working/experiment-1', 'zip', '/kaggle/working/')

ziped


In [31]:
from roboflow import Roboflow

rf = Roboflow(api_key=API_KEY)
project = rf.workspace("capstoneteam").project("road-lines-segmentation-2")
project.version(1).deploy(model_type="yolov8n", model_path="/kaggle/working/experiment-1/Segment-Model")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.97, to fix: `pip install ultralytics==8.0.196`


Would you like to continue with the wrong version of ultralytics? y/n:  y


View the status of your deployment at: https://app.roboflow.com/capstoneteam/road-lines-segmentation-2/1
Share your model with the world at: https://universe.roboflow.com/capstoneteam/road-lines-segmentation-2/model/1
